In [86]:
# Import modules
import numpy as np

# Import functions
from local_global_matrices import *

In [87]:
# Import data
d = np.fromfile('data/d.dat')
fP = np.fromfile('data/fP.dat')
fr = np.fromfile('data/fr.dat')
localK = np.fromfile('data/localK.dat')
solution = np.fromfile('data/solution.dat')

In [88]:
# Initial data
# Number of subdomains
Nsub_x = 4
Nsub_y = 3

# Number of remaining nodes in each subdomain
Nr_x = 4
Nr_y = 3

In [89]:
# Transformation matrices
# Primal nodes local-global transformation matrices
APq = create_APq_matrices(Nsub_x, Nsub_y)

# Remaining nodes local-global transformation matrices
ARr = create_ARr_matrices(Nsub_x, Nsub_y, Nr_x, Nr_y)